In [2]:
!pip install emoji==0.6.0
!pip install transformers
!pip install soynlp
!pip install pytorch_lightning
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49720 sha256=aa7be4dcf872f9112c53390893758cca6459c5aa6fba5ac4c11564e450e52f2b
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 34.

In [3]:
import os
import pandas as pd
import numpy as np
import torch
import sys
from pprint import pprint
import matplotlib.pyplot as plt
from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR
from pytorch_lightning import LightningModule, Trainer, seed_everything
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
import emoji
from soynlp.normalizer import repeat_normalize
import transformers
import emoji
import soynlp
import pytorch_lightning
from utils.emotion_model import Model
from utils.infer_emotion import infer
from utils.load_data import load_dataset, idx2emoticon, emoticon2idx, convert_sentence_emotion, add_emotion
from sentence_transformers import SentenceTransformer, util
from sklearn.datasets import make_classification
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
# from Recommend import *

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
emotion_model_path = '/content/drive/MyDrive/인공지능프로젝트/emotion_model.ckpt'
sentence_model_path = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'
emoticon_sentence_path = '/content/sentence_emoticon.txt'
idx2emoticon_path = '/content/idx2emoticon.txt'
emoticon2idx_path = '/content/emoticon2idx.txt'

In [6]:
idx2emoticon = idx2emoticon(idx2emoticon_path)
emoticon2idx = emoticon2idx(emoticon2idx_path)

In [7]:
emotion_model = Model.load_from_checkpoint(emotion_model_path)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.weight', 'classifi

In [8]:
sentence_model = SentenceTransformer(sentence_model_path)

In [305]:
emotion_weight = 1
emoticon_weight_cos = 1
find_threshold = 3
recommend_num = 5

In [306]:
emoticon_sentence, emoticon_emotions, emoticon_labels = load_dataset(emoticon_sentence_path)
emoticon_sentence_with_emotion = convert_sentence_emotion(sentence_model, emoticon_emotions, emoticon_weight_cos, emoticon_sentence)

In [307]:
emoticon_sentence_without_emotion = sentence_model.encode(emoticon_sentence)

In [308]:
print(len(emoticon_sentence_without_emotion))
print(len(emoticon_sentence_without_emotion[0]))

610
768


In [309]:
df_test = pd.read_csv('/content/cus_test.csv', sep=',')
test_sentence = df_test['sentence']
test_label = df_test['label']

In [310]:
record_hits = [] # 0, 0.5, 1, 1.5, 2
record_cos = [] 

In [311]:
test_sentence_vector = [] # n X 768
test_sentence_weighted = []                                 # n X 772
for idx in range(len(test_sentence)):
    test_sentence_vector.append(sentence_model.encode(test_sentence[idx]))
    sv_emotion = infer(emotion_model, [test_sentence[idx]]).cpu().detach().numpy()
    test_sentence_weighted.append(add_emotion(sentence_model, emotion_weight, sv_emotion, test_sentence[idx]))

In [312]:
cos_sim_weighted = cosine_similarity(test_sentence_weighted, emoticon_sentence_with_emotion)   # test_sentence X emoticon_dataset
cos_sim_only_sentence = cosine_similarity(test_sentence_vector, emoticon_sentence_without_emotion)

In [313]:
cos_sim_only_sentence[10]

array([ 0.02152853,  0.04379619,  0.21369116,  0.15318574,  0.08345224,
        0.17275536,  0.23839001,  0.13742226,  0.09561893,  0.16137855,
        0.16924196,  0.31097108,  0.26687527,  0.202762  ,  0.12558006,
        0.11679527,  0.14677398,  0.07839035,  0.05036841,  0.08964927,
        0.08414114,  0.13448963,  0.18040797,  0.3147164 ,  0.10565903,
        0.20936194,  0.19952147,  0.32120693,  0.26207787,  0.05096189,
        0.06110697, -0.0262919 ,  0.17775975,  0.12300511,  0.24625814,
        0.08599211,  0.08931283,  0.24692428,  0.27415887,  0.08390529,
        0.15123275,  0.10883506,  0.26305082,  0.13218905,  0.22208306,
        0.11163051,  0.06279806,  0.13933013,  0.23769766,  0.14370707,
        0.12696126,  0.13403562,  0.3006965 ,  0.27298462,  0.15694708,
        0.13473947,  0.1463892 ,  0.06061667,  0.19989596,  0.09515876,
        0.33558455,  0.27045885, -0.02078628, -0.11235358,  0.17803891,
        0.20620875,  0.16760948,  0.10172236,  0.16275549,  0.07

In [322]:
def recommend(idx):
    cos_rank = {i : cos_sim_weighted[idx][i] for i in range(len(emoticon_sentence_with_emotion))}
    cos_rank = sorted(cos_rank.items(), key = lambda item: item[1], reverse = True)
    selected = {i : 0 for i, _ in idx2emoticon.items()}
    selected_sentence = {i : [] for i, _ in idx2emoticon.items()}

    rec_idx = []
    ref_emotion = []
    ref_sentence = []

    found = 0
    for i in range(len(emoticon_sentence_with_emotion)):
        emoticon = emoticon_labels[cos_rank[i][0]]
        # print(cos_rank[i][1])
        selected[emoticon] += 1
        selected_sentence[emoticon].append(cos_rank[i][0])
        if selected[emoticon] == find_threshold:
            found += 1
            rec_idx.append(emoticon)
            print(idx2emoticon[emoticon])
            # print(idx2emoticon[emoticon])
            ref_emotion.append(emoticon_emotions[cos_rank[i][0]])
            # print(emoticon_emotions[cos_rank[i][0]])
            ref_sentence.append(selected_sentence[emoticon][0])
            ref_sentence.append(selected_sentence[emoticon][1])

        if found == recommend_num: break

    return rec_idx, ref_emotion, ref_sentence

In [323]:
total_hits = 0
total_cos_sim = 0

for idx in range(5000, len(test_sentence)):

    answer_emotion = test_label[idx]
    print("Answer: ", test_sentence[idx], answer_emotion)
    rec_idx, ref_emotion, ref_sentence = recommend(idx)
    print(ref_emotion)
    total_hits += np.sum(np.array(ref_emotion) == answer_emotion)
    total_cos_sim += np.sum(np.array(cos_sim_only_sentence[idx][ref_sentence]))
    print(np.array(cos_sim_weighted[idx][ref_sentence]))
    print(np.array(cos_sim_only_sentence[idx][ref_sentence]))

    if idx == 5009: break
hit_recommend_num = total_hits / (recommend_num * len(test_sentence))
avg_cos_sim = total_cos_sim / (find_threshold * recommend_num * len(test_sentence))

record_hits.append(hit_recommend_num)
record_cos.append(avg_cos_sim)

Answer:  금번에 대학교 원서 쓰는데 부모님 의사에 따르지 않고 내뜻대로 써버렸어. 1
( `ε´ )
||｀Д´)ﾉ
͡ ͜ʖ ͡ ╭∩╮
ಠ ɞ ಠ
(╬ Ò ‸ Ó)
[3, 3, 3, 3, 3]
[0.64028954 0.63039077 0.60264117 0.59719007 0.6030331  0.59559161
 0.6172565  0.59445028 0.59115971 0.58113732]
[0.39487737 0.3750799  0.31958067 0.30867845 0.32036456 0.30548152
 0.34881133 0.3031989  0.29661775 0.2765729 ]
Answer:  준비해왔던 결혼식을 미루거나 취소해야 할 것 같아 너무 슬퍼. 0
。：゜(；´∩｀；)゜：。
(ఠ్ఠ ˓̭ ఠ్ఠ)
(ू˃̣̣̣̣̣̣︿˂̣̣̣̣̣̣ ू)
( ᴗ_ᴗ̩̩ )
(oT-T)尸
[0, 0, 0, 0, 0]
[0.75986965 0.75367525 0.77167488 0.74308325 0.74359711 0.7029326
 0.77910005 0.73854044 0.75067784 0.69628822]
[0.5200472  0.50765836 0.54365754 0.4864744  0.4875021  0.40617317
 0.5585079  0.47738868 0.5016636  0.39288446]
Answer:  다른 사람들은 내 돈이 부럽다고 하는데 나는 몸이 안 좋아서 행복하지 않아. 0
o(╥﹏╥)o
。：゜(；´∩｀；)゜：。
(☍﹏⁰)
(ఠ్ఠ ˓̭ ఠ్ఠ)
( -̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥᷄◞ω◟-̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥̥᷅ )
[0, 0, 0, 0, 0]
[0.71337427 0.64299554 0.6228692  0.60141484 0.5934456  0.58391857
 0.60092414 0.57956006 0.73364486 0.58001996]
[0.49968553 0.35892808 0

In [317]:
print(hit_recommend_num)
print(avg_cos_sim)

0.6945622119815669
0.23927060605102604


In [336]:
def recommend_emoticon(input_sentence):

    input_emotion = infer(emotion_model, [input_sentence]).cpu().detach().numpy()
    input_vector = add_emotion(sentence_model, emotion_weight, input_emotion, input_sentence)
    input_cos_sim = cosine_similarity([input_vector], emoticon_sentence_with_emotion)

    cos_rank = {i : input_cos_sim[0][i] for i in range(len(emoticon_sentence_with_emotion))}
    cos_rank = sorted(cos_rank.items(), key = lambda item: item[1], reverse = True)
    selected = {i : 0 for i, _ in idx2emoticon.items()}
    selected_sentence = {i : [] for i, _ in idx2emoticon.items()}

    rec_idx = []
    ref_emotion = []
    ref_sentence = []

    found = 0
    for i in range(len(emoticon_sentence_with_emotion)):
        emoticon = emoticon_labels[cos_rank[i][0]]
        # print(cos_rank[i][1])
        selected[emoticon] += 1
        selected_sentence[emoticon].append(cos_rank[i][0])
        if selected[emoticon] == find_threshold:
            found += 1
            rec_idx.append(emoticon)
            print(idx2emoticon[emoticon])
            # print(idx2emoticon[emoticon])
            ref_emotion.append(emoticon_emotions[cos_rank[i][0]])
            # print(emoticon_emotions[cos_rank[i][0]])
            ref_sentence.append(selected_sentence[emoticon][0])
            ref_sentence.append(selected_sentence[emoticon][1])

        if found == recommend_num: break

In [341]:
recommend_emoticon('아 짜증난다 나랑 싸울래?')

୯ૃ(⁎⁾̵ ਊ ⁽̵)੭ુ⁼³₌₃
o(｀ᾥ´)o
⁽⁽(੭ꐦ •̀Д•́ )੭*⁾⁾
(ʘ言ʘ╬)
(ง︡'-'︠)ง
